In [3]:
from IPython.display import display
from sympy import *
import numpy as np
init_printing()

def gradient(scalar_function, variables):
    matrix_scalar_function = Matrix([scalar_function])
    return matrix_scalar_function.jacobian(variables)

In [4]:
class TriBasis():
    def __init__(self, p):
        self.p = p
        self.idx_matrix()
        
    def idx_matrix(self):
        p = self.p
        ldof = self.number_of_basis_function()
        idx = Matrix(range(ldof))
        f = lambda x: floor((-1 + sqrt(1 + 8*x))/2)
        g = lambda x: x*(x +1)/2
        idx0 = idx.applyfunc(f)
        idxMatrix = zeros(ldof, 3)
        idxMatrix[:,2] = idx - idx0.applyfunc(g)
        idxMatrix[:,1] = idx0 - idxMatrix[:,2]
        idxMatrix[:,0] = ones(ldof, 1)*p - idxMatrix[:, 1] - idxMatrix[:, 2] 
        self.idxMatrix = idxMatrix
        
    def basis(self, lam0, lam1, lam2):
        p = self.p
        ldof = self.number_of_basis_function()
        phi = zeros(ldof, 1)
        t = Matrix(range(p))/p
        f = lambda x: 1/factorial(x)
        bc = Matrix([lam0, lam1, lam2])
        P = ones(p+1, 1)
        c = Matrix(range(0, p+1))
        P = c.applyfunc(f)
        AA = ones(p+1, 3)
        AA[1:, :] = ones(p, 1)*bc.transpose() - t*ones(1, 3)
        # B = DiagonalMatrix(P)*Matrix(np.multiply.accumulate(AA, axis=0))
        B = Matrix(np.multiply.accumulate(AA, axis=0))
        idxMatrix = self.idxMatrix
        for i in range(ldof):
            phi[i] = B[idxMatrix[i, 0], 0]*B[idxMatrix[i, 1], 1]*B[idxMatrix[i, 2], 2]
        return phi
        
    def number_of_basis_function(self):
        p = self.p
        return int((p+1)*(p+2)/2)

In [5]:
x, y, x0, y0, x1, y1, x2, y2, theta = symbols('x y x_0 y_0 x_1 y_1 x_2 y_2 theta')

In [6]:
p = Matrix([[x, y]])
p0 = Matrix([[x0, y0]])
p1 = Matrix([[x1, y1]])
p2 = Matrix([[x2, y2]])

In [7]:
W = Matrix([[0,1], [-1, 0]])
A = Matrix([
        [x0, x1, x2],
        [y0, y1, y2],
        [1, 1, 1]
    ])
D0 = Matrix([
        [x, x1, x2],
        [y, y1, y2],
        [1, 1, 1]
    ])
D1 = Matrix([
        [x0, x, x2],
        [y0, y, y2],
        [1, 1, 1]
    ])
D2 = Matrix([
        [x0, x1, x],
        [y0, y1, y],
        [1, 1, 1]
    ])

In [8]:
lam0 = det(D0)/det(A)
lam1 = det(D1)/det(A)
lam2 = det(D2)/det(A)

In [16]:
phi0 = simplify(lam1*gradient(lam2, p)*W.transpose() - lam2*gradient(lam1, p)*W.transpose())
phi1 = simplify(lam2*gradient(lam0, p)*W.transpose() - lam0*gradient(lam2, p)*W.transpose())
phi2 = simplify(lam0*gradient(lam1, p)*W.transpose() - lam1*gradient(lam0, p)*W.transpose())

## 验证 
f = simplify((p2 - p1)*W.transpose()*phi0.transpose())
f = simplify(f.subs({x:x1 + theta*(x2 - x1), y:y1+theta*(y2 - y1)}))
display(f)

g = simplify((p0 - p2)*W.transpose()*phi0.transpose())
g = simplify(g.subs({x:x2 + theta*(x0 - x2), y:y2+theta*(y0 - y2)}))
display(g)

m = simplify((p1 - p0)*W.transpose()*phi0.transpose())
m = simplify(m.subs({x:x0 + theta*(x1 - x0), y:y0+theta*(y1 - y0)}))
display(m)

In [10]:
psi0 = simplify(lam1*gradient(lam2, p)*W.transpose() + lam2*gradient(lam1, p)*W.transpose())
psi1 = simplify(lam2*gradient(lam0, p)*W.transpose() + lam0*gradient(lam2, p)*W.transpose())
psi2 = simplify(lam0*gradient(lam1, p)*W.transpose() + lam1*gradient(lam0, p)*W.transpose())
psi = simplify((p2 - p1)*W.transpose()*psi0.transpose())[0]*(lam1 - lam2)
psi = simplify(psi.subs({x:x1 + theta*(x2 - x1), y:y1+theta*(y2 - y1)}))
d = Integral(psi, (theta, 0, 1))
display(d.nsimplify())

In [11]:
k = 1
triBasis = TriBasis(k)
b = triBasis.basis(lam0, lam1, lam2)
b = b[-1:-(k+2):-1]
for i in range(k+1):
    bi = simplify(b[i].subs({x:x1 + theta*(x2 - x1), y:y1+theta*(y2 - y1)}))
    d = Integral(bi, (theta, 0, 1))
    display(d.nsimplify())
    

In [12]:
1, 1 /2
1, 2, 1/12
2, 2, 2, 2/72
7, 8, 2, 8, 7/960

In [13]:
factorial(3)*2

## Nedelec's edge element 

### 2d 

### 3d 